In [1]:
import os
import re
import time
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

In [ ]:
df = pd.read_csv('new_tourism_with_id.csv')
df.head()

,Place_Id,Place_Name,Description,Category,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Unnamed: 11,Unnamed: 12
0,1,Monumen Nasional,Monumen Nasional atau yang populer disingkat d...,Budaya,Jakarta,20000,4.6,15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,NaN,1
1,2,Kota Tua,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Budaya,Jakarta,0,4.6,90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,NaN,2
2,3,Dunia Fantasi,Dunia Fantasi atau disebut juga Dufan adalah t...,Taman Hiburan,Jakarta,270000,4.6,360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,NaN,3
3,4,Taman Mini Indonesia Indah (TMII),Taman Mini Indonesia Indah merupakan suatu kaw...,Taman Hiburan,Jakarta,10000,4.5,NaN,"{'lat': -6.302445899999999, 'lng': 106.8951559}",-6.302446,106.895156,NaN,4
4,5,Atlantis Water Adventure,Atlantis Water Adventure atau dikenal dengan A...,Taman Hiburan,Jakarta,94000,4.5,60.0,"{'lat': -6.12419, 'lng': 106.839134}",-6.124190,106.839134,NaN,5


In [9]:
keywords = df['Place_Name'].copy().tolist()
print(len(keywords), keywords[:2], '...', keywords[-2:])

437 ['Monumen Nasional', 'Kota Tua'] ... ['Taman Flora Bratang Surabaya', 'Gereja Perawan Maria Tak Berdosa Surabaya']


In [21]:
def search_google(keyword: str, num:int, driver: webdriver):

    # Pastikan nama keyword tidak ada tanda "\" atau "/"
    keyword = keyword.replace('\\', ' ').replace('/', ' ')
    
    img_xpath = '/html/body/div[11]/div[2]/div[3]/div/div/c-wiz/div/div[2]/div[2]/div/div[2]/c-wiz/div/div[3]/div[1]/a/img[1]'
    search_url = f"https://www.google.com/search?tbm=isch&q={keyword}"
    img_output_dir = 'images_output'
    csv_output_dir = 'csv_output'
    data = list()
    counter = 0

    if not os.path.exists(csv_output_dir):
        os.makedirs(csv_output_dir)
    if not os.path.exists(img_output_dir):
        os.makedirs(img_output_dir)
    if not os.path.exists(os.path.join(img_output_dir, keyword)):
        os.makedirs(os.path.join(img_output_dir, keyword))
    
    driver.get(search_url)
    time.sleep(3)

    imgs_pointer = driver.find_elements(By.CSS_SELECTOR,'div.F0uyec')
    
    for img_button in imgs_pointer:
        
        # If the number of image requests is met, then stop.
        if len(data) == num:
            break

        # Click the image, to display the original image.
        driver.execute_script("arguments[0].click();", img_button)

        # Give 2 chances to get the original image link,
        # rather than the cached version of the image.
        for _ in range(2):
            time.sleep(2)
            img_link = driver.find_element(By.XPATH, img_xpath).get_attribute('src')
            cached_img = img_link.startswith('https://encrypted-tbn0.gstatic.com/')
            jpg_png = re.search(r'\.(jpg|png).*$', img_link)
            if not cached_img and jpg_png:  # If the image is not a cache image, then save the link.
                break
        
        # If still only get the cache image, then go to the next image.
        if cached_img or not jpg_png:
            continue

        # Coba download
        try:
            img_data = requests.get(img_link).content
            img_path = os.path.join(img_output_dir, keyword, f'{keyword}_{counter}.{jpg_png[1]}')
            with open(img_path, 'wb') as handler:
                handler.write(img_data)
        except Exception as e:
            print(e)
            continue
        
        # Append the link to data
        data.append({
            'keyword': keyword,
            'link': img_link
        })

        print('  ' + img_link)
        counter += 1

    # Save all the links to CSV file
    temp_df = pd.DataFrame(data)
    csv_path = os.path.join(csv_output_dir, f'{keyword}.csv')
    temp_df.to_csv(csv_path, index=False)
    print(f"  Successfully downloaded {len(data)} {keyword} images")


In [29]:
keywords_start_316 = keywords[316:]
len(keywords_start_316)

121

In [30]:
# keywords = ['Monumen Nasional', 'Kota Tua']
num_of_image = 3
driver = webdriver.Firefox()
driver.get('https://google.com')
    
# running the seach function that we created above through the keywords
for index, keyword in enumerate(keywords_start_316):
    print(f"{index + 316} : Scraping {keyword} images")
    search_google(keyword=keyword, num=num_of_image, driver=driver)

driver.close()


316 : Scraping Curug Cimahi images
  https://www.king-adventure.com/wp-content/uploads/2020/11/curug-cimahi.jpg
  https://assets.promediateknologi.id/crop/0x0:0x0/750x500/webp/photo/2022/05/21/2450950846.jpg
  https://nyaringindonesia.com/wp-content/uploads/2024/06/IMG_20240630_162015.jpg
  Successfully downloaded 3 Curug Cimahi images
317 : Scraping Situ Patenggang images
  https://wonderfulimage.s3-id-jkt-1.kilatstorage.id/1716201730-drone-8-jpg-medium.jpg
  https://ik.imagekit.io/tvlk/blog/2018/08/Bandung-Situ-Patenggang-1126603460.jpg
  https://img.okezone.com/content/2023/02/16/408/2766122/situ-patenggang-lokasi-fasilitas-dan-harga-tiket-masuk-MHTGT5EN2K.jpg
  Successfully downloaded 3 Situ Patenggang images
318 : Scraping Kawah Rengganis Cibuni images
  https://asset.kompas.com/crops/2WG1L4Qi0lkyPCKbGWu0O81RdK8=/24x0:869x563/1200x800/data/photo/2022/03/15/62307d3632949.jpg
  https://vilamurahciwidey.com/wp-content/uploads/2023/01/Kawah-engganis-Ciwidey.jpg
  https://ik.imagekit.i

In [ ]:
input_string = 'Desa Wisata Rumah Domes/Teletubbies'
input_string = 'Desa Wisata Rumah Domes Teletubbies'
input_string = input_string.replace("\\", "").replace("/", " ")
input_string

'Desa Wisata Rumah Domes afas Teletubbies'

Hilangkan tanda "\\" dan "/" dari seluruh nama wisata di dataset